Get Data

In [14]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import requests

In [15]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url, headers=header)

tables = pd.read_html(r.text)

df=pd.DataFrame(tables[0])

# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

df.columns=['Postcode','Borough','Neighbourhood']

df.drop([0],axis=0,inplace=True)

df.reset_index()

# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)

# More than one neighborhood can exist in one postal code area. 
# For example, in the table on the Wikipedia page, 
# you will notice that M5A is listed twice and has two neighborhoods: 
# Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods 
# separated with a comma as shown in row 11 in the above table.

df1=df.groupby("Postcode").agg(lambda x:','.join(set(x)))



In [16]:
# If a cell has a borough but a Not assigned neighborhood, 
# then the neighborhood will be the same as the borough. 
# So for the 9th cell in the table on the Wikipedia page, 
# the value of the Borough and the Neighborhood columns will be Queen's Park.

df1.loc[df1['Neighbourhood']=="Not assigned",'Neighbourhood']=df1.loc[df1['Neighbourhood']=="Not assigned",'Borough']

#df2.head()

In [18]:
df1.head(20)

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Malvern,Rouge"
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
M1E,Scarborough,"Morningside,West Hill,Guildwood"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"Kennedy Park,Ionview,East Birchmount Park"
M1L,Scarborough,"Oakridge,Golden Mile,Clairlea"
M1M,Scarborough,"Scarborough Village West,Cliffside,Cliffcrest"


In [20]:
df2 = df1.reset_index()
df2.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Morningside,West Hill,Guildwood"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [21]:
#Now we can remove the duplicate boroughts as follows:
df2['Borough']= df2['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")

In [23]:
df2.head()
df2.shape

(103, 3)

# QUESTION 2:  Getting the geo coordinates

In [31]:
pip install geopy


The following command must be run outside of the IPython shell:

    $ pip install geopy

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [32]:
from  geopy.geocoders import Nominatim
geolocator = Nominatim()
city ="London"
country ="Uk"
loc = geolocator.geocode(city+','+ country)
print("latitude is :-" ,loc.latitude,"\nlongtitude is:-" ,loc.longitude)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


latitude is :- 51.5073219 
longtitude is:- -0.1276474


In [33]:
from  geopy.geocoders import Nominatim
geolocator = Nominatim()
location = geolocator.geocode("Toronto, North York, Parkwoods")

print(location.address)
print('')
print((location.latitude, location.longitude))
print('')
print(location.raw)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


Parkwoods Village Drive, Parkway East, Don Valley East, North York, Toronto, Ontario, M3A 2X2, Canada

(43.7587999, -79.3201966)

{'place_id': 112665056, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 160406961, 'boundingbox': ['43.7576231', '43.761106', '-79.3239088', '-79.316215'], 'lat': '43.7587999', 'lon': '-79.3201966', 'display_name': 'Parkwoods Village Drive, Parkway East, Don Valley East, North York, Toronto, Ontario, M3A 2X2, Canada', 'class': 'highway', 'type': 'secondary', 'importance': 0.51}


In [34]:
import pandas as pd
df_geopy = pd.DataFrame({'PostalCode': ['M3A', 'M4A', 'M5A'],
                         'Borough': ['North York', 'North York', 'Downtown Toronto'],
                         'Neighbourhood': ['Parkwoods', 'Victoria Village', 'Harbourfront'],})

from geopy.geocoders import Nominatim
geolocator = Nominatim()


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:7: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


In [38]:
df_geopy1 = df2
#df_geopy1

In [39]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()

df_geopy1['address'] = df2[['Postcode', 'Borough', 'Neighbourhood']].apply(lambda x: ', '.join(x), axis=1 )
df_geopy1.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


,Postcode,Borough,Neighbourhood,address
0,M1B,Scarborough,"Malvern,Rouge","M1B, Scarborough, Malvern,Rouge"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union","M1C, Scarborough, Highland Creek,Rouge Hill,Po..."
2,M1E,Scarborough,"Morningside,West Hill,Guildwood","M1E, Scarborough, Morningside,West Hill,Guildwood"
3,M1G,Scarborough,Woburn,"M1G, Scarborough, Woburn"
4,M1H,Scarborough,Cedarbrae,"M1H, Scarborough, Cedarbrae"


In [40]:
df_geopy1.shape

(103, 4)

In [41]:
df_geopy1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 4 columns):
Postcode         103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
address          103 non-null object
dtypes: object(4)
memory usage: 3.3+ KB


In [42]:
df_geopy1.drop(df_geopy1[df_geopy1['Borough']=="Notassigned"].index,axis=0, inplace=True)
#df_geopy1
# code holds true up until i=102
df_geopy1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 4 columns):
Postcode         103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
address          103 non-null object
dtypes: object(4)
memory usage: 4.0+ KB


In [43]:
#df_geopy1.head()

In [44]:
df_geopy1.shape

(103, 4)

In [45]:
df_geopy1.to_csv('geopy1.csv')

In [46]:
#testing data
from  geopy.geocoders import Nominatim
geolocator = Nominatim()
location = geolocator.geocode("M1G, Scarborough, Woburn")


print(location.address)

print((location.latitude, location.longitude))

print(location.raw)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


Woburn, Scarborough—Guildwood, Scarborough, Toronto, Golden Horseshoe, Ontario, M1H 2A2, Canada
(43.7598243, -79.2252908)
{'place_id': 4972888, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'node', 'osm_id': 558715617, 'boundingbox': ['43.7597743', '43.7598743', '-79.2253408', '-79.2252408'], 'lat': '43.7598243', 'lon': '-79.2252908', 'display_name': 'Woburn, Scarborough—Guildwood, Scarborough, Toronto, Golden Horseshoe, Ontario, M1H 2A2, Canada', 'class': 'place', 'type': 'neighbourhood', 'importance': 0.558803667479001}


In [47]:
pip install geocoder


The following command must be run outside of the IPython shell:

    $ pip install geocoder

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [49]:
df2.to_csv('geopy.csv')

In [50]:
import csv

with open('geopy.csv') as csvfile:
     reader = csv.DictReader(csvfile)
     #for row in reader:
         #print(row['Postcode'],row['Borough'], row['Neighbourhood'] )

In [51]:
from  geopy.geocoders import Nominatim
geolocator = Nominatim()
location = geolocator.geocode("M1B Scarborough Rouge,Malvern")

print(location.address)

print((location.latitude, location.longitude))

print(location.raw)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


Malvern, Scarborough—Rouge Park, Scarborough, Toronto, Golden Horseshoe, Ontario, M1B 4Y7, Canada
(43.8091955, -79.2217008)
{'place_id': 4979177, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'node', 'osm_id': 558715616, 'boundingbox': ['43.8091455', '43.8092455', '-79.2217508', '-79.2216508'], 'lat': '43.8091955', 'lon': '-79.2217008', 'display_name': 'Malvern, Scarborough—Rouge Park, Scarborough, Toronto, Golden Horseshoe, Ontario, M1B 4Y7, Canada', 'class': 'place', 'type': 'neighbourhood', 'importance': 0.769448365439565}


In [52]:
pip install geopy


The following command must be run outside of the IPython shell:

    $ pip install geopy

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [53]:
from geopy.geocoders import Nominatim
print('Nominatim imported')

Nominatim imported


In [57]:
df_geopy.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront


In [58]:
df_geopy['address']=df_geopy['PostalCode'] + ',' + df_geopy['Borough'] + ','+ df_geopy['Neighbourhood']

In [59]:
df_geopy.head()

,PostalCode,Borough,Neighbourhood,address
0,M3A,North York,Parkwoods,"M3A,North York,Parkwoods"
1,M4A,North York,Victoria Village,"M4A,North York,Victoria Village"
2,M5A,Downtown Toronto,Harbourfront,"M5A,Downtown Toronto,Harbourfront"


In [60]:
nom = Nominatim()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  if __name__ == '__main__':


In [61]:
df_geopy['Coordinates'] =df_geopy['address'].apply(nom.geocode)
df_geopy.head()

,PostalCode,Borough,Neighbourhood,address,Coordinates
0,M3A,North York,Parkwoods,"M3A,North York,Parkwoods","(Parkwoods Village Drive, Parkway East, Don Va..."
1,M4A,North York,Victoria Village,"M4A,North York,Victoria Village",None
2,M5A,Downtown Toronto,Harbourfront,"M5A,Downtown Toronto,Harbourfront",None


In [62]:
df_geopy['latitude']=df_geopy['Coordinates'].apply(lambda x: x.latitude if x !=None else None)
df_geopy['longitude']=df_geopy['Coordinates'].apply(lambda x: x.longitude if x !=None else None)
df_geopy.head()

,PostalCode,Borough,Neighbourhood,address,Coordinates,latitude,longitude
0,M3A,North York,Parkwoods,"M3A,North York,Parkwoods","(Parkwoods Village Drive, Parkway East, Don Va...",43.761224,-79.323986
1,M4A,North York,Victoria Village,"M4A,North York,Victoria Village",None,NaN,NaN
2,M5A,Downtown Toronto,Harbourfront,"M5A,Downtown Toronto,Harbourfront",None,NaN,NaN


In [63]:
df_geopy.to_csv('geo_loc_py.csv')

In [64]:
print('The latitude of', df_geopy.address[0],  'is', df_geopy.latitude[0], 'and its longitude is',df_geopy.longitude[0])


The latitude of M3A,North York,Parkwoods is 43.7612239 and its longitude is -79.3239857


In [65]:
import pandas as pd 
# Read data from file 'filename.csv' 
# (in the same directory that your python process is based)
# Control delimiters, rows, column names with read_csv (see later) 
data2 = pd.read_csv("geopy.csv") 
# Preview the first 5 lines of the loaded data 
data2.head()

,Unnamed: 0,Postcode,Borough,Neighbourhood,address
0,0,M1B,Scarborough,"Malvern,Rouge","M1B, Scarborough, Malvern,Rouge"
1,1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union","M1C, Scarborough, Highland Creek,Rouge Hill,Po..."
2,2,M1E,Scarborough,"Morningside,West Hill,Guildwood","M1E, Scarborough, Morningside,West Hill,Guildwood"
3,3,M1G,Scarborough,Woburn,"M1G, Scarborough, Woburn"
4,4,M1H,Scarborough,Cedarbrae,"M1H, Scarborough, Cedarbrae"


In [73]:
data3 = pd.read_csv("http://cocl.us/Geospatial_data") 
# Preview the first 5 lines of the loaded data 
data3.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [75]:
data3.rename(columns={'Postal Code': 'Postcode'}, inplace=True)
data3.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [76]:
data1 = pd.merge(data3, data2, how='inner', on=None, left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

data1.head()

,Postcode,Latitude,Longitude,Unnamed: 0,Borough,Neighbourhood,address
0,M1B,43.806686,-79.194353,0,Scarborough,"Malvern,Rouge","M1B, Scarborough, Malvern,Rouge"
1,M1C,43.784535,-79.160497,1,Scarborough,"Highland Creek,Rouge Hill,Port Union","M1C, Scarborough, Highland Creek,Rouge Hill,Po..."
2,M1E,43.763573,-79.188711,2,Scarborough,"Morningside,West Hill,Guildwood","M1E, Scarborough, Morningside,West Hill,Guildwood"
3,M1G,43.770992,-79.216917,3,Scarborough,Woburn,"M1G, Scarborough, Woburn"
4,M1H,43.773136,-79.239476,4,Scarborough,Cedarbrae,"M1H, Scarborough, Cedarbrae"


In [77]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 7 columns):
Postcode         103 non-null object
Latitude         103 non-null float64
Longitude        103 non-null float64
Unnamed: 0       103 non-null int64
Borough          103 non-null object
Neighbourhood    103 non-null object
address          103 non-null object
dtypes: float64(2), int64(1), object(4)
memory usage: 6.4+ KB


In [78]:
cols = data1.columns.tolist()
cols

['Postcode',
 'Latitude',
 'Longitude',
 'Unnamed: 0',
 'Borough',
 'Neighbourhood',
 'address']

In [79]:
new_column_order = ['Postcode',
 'Borough',
 'Neighbourhood',
 'Latitude',
 'Longitude']
new_column_order

['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']

In [80]:
data1 = data1[new_column_order]
sorted_df = data1.sort_values([ 'Neighbourhood', 'Latitude'], ascending=[True, True])

In [81]:
sorted_df.reset_index(inplace=True)
sorted_df.head()

,index,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,12,M1S,Scarborough,Agincourt,43.794200,-79.262029
1,89,M8W,Etobicoke,"Alderwood,Long Branch",43.602414,-79.543484
2,68,M5V,DowntownToronto,"Bathurst Quay,Harbourfront West,Island airport...",43.628947,-79.394420
3,19,M2K,NorthYork,Bayview Village,43.786947,-79.385975
4,56,M5E,DowntownToronto,Berczy Park,43.644771,-79.373306


In [82]:
sorted_cols =sorted_df.columns.tolist()
new_column_order2 = ['Postcode',
 'Borough',
 'Neighbourhood',
 'Latitude',
 'Longitude']
new_column_order2

['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']

In [83]:
sorted_dataframe = sorted_df[new_column_order]
sorted_dataframe.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1S,Scarborough,Agincourt,43.794200,-79.262029
1,M8W,Etobicoke,"Alderwood,Long Branch",43.602414,-79.543484
2,M5V,DowntownToronto,"Bathurst Quay,Harbourfront West,Island airport...",43.628947,-79.394420
3,M2K,NorthYork,Bayview Village,43.786947,-79.385975
4,M5E,DowntownToronto,Berczy Park,43.644771,-79.373306


In [84]:
sorted_dataframe.to_csv('sorted_geoloc.csv')

# Question 3 

# Build a test set with boroughs in Toronto

In [86]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans



In [87]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.3.1               |             py_0          25 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    altair-4.0.0               |             py_0         606 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following NEW packages will be 

In [88]:
sorted_dataframe.info()
sorted_dataframe.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 5 columns):
Postcode         103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
Latitude         103 non-null float64
Longitude        103 non-null float64
dtypes: float64(2), object(3)
memory usage: 4.1+ KB


(103, 5)

In [89]:
sorted_dataframe.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1S,Scarborough,Agincourt,43.794200,-79.262029
1,M8W,Etobicoke,"Alderwood,Long Branch",43.602414,-79.543484
2,M5V,DowntownToronto,"Bathurst Quay,Harbourfront West,Island airport...",43.628947,-79.394420
3,M2K,NorthYork,Bayview Village,43.786947,-79.385975
4,M5E,DowntownToronto,Berczy Park,43.644771,-79.373306


In [90]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(sorted_dataframe['Borough'].unique()),
        sorted_dataframe.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [91]:
!conda install -c conda-forge folium=0.5.0 

Solving environment: done

# All requested packages already installed.



In [92]:
import pandas as pd
import folium

print('imported pandas & folium')

imported pandas & folium


In [95]:
import pandas as pd
import folium

#grab a random sample from df
subset_of_df = sorted_dataframe.sample(n=11)
map_test = folium.Map(location=[subset_of_df['Latitude'].mean(), 
                                subset_of_df['Longitude'].mean()], 
                      zoom_start=10)
#creating a Marker for each point in df_sample. Each point will get a popup with their zip
for row in subset_of_df.itertuples(): 
    map_test.add_child(folium.Marker(location=[row.Latitude ,row.Longitude],
           popup=row.Borough))


In [96]:

#open map_test.html in browser
map_test.save("map_test.html")

In [97]:
from folium.plugins import MarkerCluster
map_borough = folium.Map(location=[subset_of_df['Latitude'].mean(), 
 subset_of_df['Longitude'].mean()], 
 zoom_start=10)
mc = MarkerCluster()
#creating a Marker for each point in df_sample. Each point will get a popup with their zip
for row in subset_of_df.itertuples():
    mc.add_child(folium.Marker(location=[row.Latitude,  row.Longitude],
                 popup=row.Borough))
    map_borough.add_child(mc)


#map_borough

#open in map_borough.html browser 
map_borough.save("map_borough.html")


In [98]:
import pandas as pd
import folium



#grab a random sample from df
toronto_n = sorted_dataframe.sample(n=20)
map_toronto = folium.Map(location=[toronto_n['Latitude'].mean(), 
                                toronto_n['Longitude'].mean()], 
                      zoom_start=10)
#creating a Marker for each point in df_sample. Each point will get a popup with their zip
for row in toronto_n.itertuples():
    map_toronto.add_child(folium.Marker(location=[row.Latitude ,row.Longitude],
           popup=row.Neighbourhood))

    
map_toronto 

#open map_toronto.html in browser

map_toronto.save("map_toronto20.html")



In [99]:
sorted_dataframe.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1S,Scarborough,Agincourt,43.794200,-79.262029
1,M8W,Etobicoke,"Alderwood,Long Branch",43.602414,-79.543484
2,M5V,DowntownToronto,"Bathurst Quay,Harbourfront West,Island airport...",43.628947,-79.394420
3,M2K,NorthYork,Bayview Village,43.786947,-79.385975
4,M5E,DowntownToronto,Berczy Park,43.644771,-79.373306


In [102]:
address = 'Toronto, CA'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [103]:
# create map of Toronto using latitude and longitude values
map_toronto_neighbourhoods = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(sorted_dataframe['Latitude'], sorted_dataframe['Longitude'], sorted_dataframe['Borough'], sorted_dataframe['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_neighbourhoods)  
    
map_toronto_neighbourhoods

map_toronto_neighbourhoods.save("map_toronto_neighbourhoods.html")

In [104]:
address = 'York, Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of York, Toronto are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinates of York, Toronto are 43.67910515, -79.4911841400715.


In [105]:
york_data = sorted_dataframe[sorted_dataframe['Borough'] == 'York'].reset_index(drop=True)
york_data

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512
1,M6M,York,"Del Ray,Mount Dennis,Keelesdale,Silverthorn",43.691116,-79.476013
2,M6C,York,Humewood-Cedarvale,43.693781,-79.428191
3,M6N,York,"Runnymede,The Junction North",43.673185,-79.487262
4,M9N,York,Weston,43.706876,-79.518188


In [106]:
york_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
Postcode         5 non-null object
Borough          5 non-null object
Neighbourhood    5 non-null object
Latitude         5 non-null float64
Longitude        5 non-null float64
dtypes: float64(2), object(3)
memory usage: 280.0+ bytes


In [107]:
# create map of Manhattan using latitude and longitude values
map_york_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(york_data['Latitude'], york_data['Longitude'], york_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_york_toronto)  
    
map_york_toronto

map_york_toronto.save("map_york_toronto.html")


In [110]:
neighbourhood_latitude = york_data.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = york_data.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = york_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Caledonia-Fairbanks are 43.6890256, -79.453512.


In [118]:
LIMIT = 100

radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(

'Y0CI0AIJ2CPC5OUAHVYKRSF22QSKULBWOXHLAODLJGSEM13X', 

'C0ILED4TRBBOZY1GZCRELHRNCFCWX1FO2CLBRYZ4VGKKD1WD', 

'20191225', 

neighbourhood_latitude, 

neighbourhood_longitude, 

radius, 

LIMIT)

url



'https://api.foursquare.com/v2/venues/explore?&client_id=Y0CI0AIJ2CPC5OUAHVYKRSF22QSKULBWOXHLAODLJGSEM13X&client_secret=C0ILED4TRBBOZY1GZCRELHRNCFCWX1FO2CLBRYZ4VGKKD1WD&v=20191225&ll=43.6890256,-79.453512&radius=500&limit=100'

In [120]:
york_results = requests.get(url).json()
york_results

{'meta': {'code': 200, 'requestId': '5e03ebdf1d67cb001b9557d6'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 43.6935256045, 'lng': -79.44730040297749},
   'sw': {'lat': 43.6845255955, 'lng': -79.45972359702252}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c05742c517d0f472d90f415',
       'name': 'KFC',
       'location': {'address': '2296 Eglinton Avenue West',
        'lat': 43.69064732256945,
        'lng': -79.456325832418,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.69064732256945,
          'lng': -79.456

In [121]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [122]:
york_venues = york_results['response']['groups'][0]['items']
    
york_nearby_venues = json_normalize(york_venues) # flatten JSON

# filter columns
york_filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
york_nearby_venues = york_nearby_venues.loc[:, york_filtered_columns]

# filter the category for each row
york_nearby_venues['venue.categories'] = york_nearby_venues.apply(get_category_type, axis=1)

# clean columns
york_nearby_venues.columns = [col.split(".")[-1] for col in york_nearby_venues.columns]

york_nearby_venues.head()

,name,categories,lat,lng
0,KFC,Fast Food Restaurant,43.690647,-79.456326
1,Nairn Park,Park,43.690654,-79.456300
2,Maximum Woman,Women's Store,43.690651,-79.456333
3,Walmart,Market,43.690660,-79.456317
4,Fairbank Memorial Park,Park,43.692028,-79.448924


In [123]:
print('{} venues were returned by Foursquare.'.format(york_nearby_venues.shape[0]))

5 venues were returned by Foursquare.


In [124]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            'Y0CI0AIJ2CPC5OUAHVYKRSF22QSKULBWOXHLAODLJGSEM13X', 

'C0ILED4TRBBOZY1GZCRELHRNCFCWX1FO2CLBRYZ4VGKKD1WD', 

'20191225', 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [125]:
york_venues = getNearbyVenues(names=york_data['Neighbourhood'],
                                   latitudes=york_data['Latitude'],
                                   longitudes=york_data['Longitude']
                                  )

Caledonia-Fairbanks
Del Ray,Mount Dennis,Keelesdale,Silverthorn
Humewood-Cedarvale
Runnymede,The Junction North
Weston


In [126]:
york_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Caledonia-Fairbanks,43.689026,-79.453512,KFC,43.690647,-79.456326,Fast Food Restaurant
1,Caledonia-Fairbanks,43.689026,-79.453512,Nairn Park,43.690654,-79.456300,Park
2,Caledonia-Fairbanks,43.689026,-79.453512,Maximum Woman,43.690651,-79.456333,Women's Store
3,Caledonia-Fairbanks,43.689026,-79.453512,Walmart,43.690660,-79.456317,Market
4,Caledonia-Fairbanks,43.689026,-79.453512,Fairbank Memorial Park,43.692028,-79.448924,Park


In [127]:
york_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Caledonia-Fairbanks,5,5,5,5,5,5
"Del Ray,Mount Dennis,Keelesdale,Silverthorn",5,5,5,5,5,5
Humewood-Cedarvale,3,3,3,3,3,3
"Runnymede,The Junction North",4,4,4,4,4,4
Weston,1,1,1,1,1,1


In [128]:
print('There are {} uniques categories.'.format(len(york_venues['Venue Category'].unique())))

There are 14 uniques categories.


In [129]:
york_onehot = pd.get_dummies(york_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
york_onehot['Neighbourhood'] = york_venues['Neighbourhood'] 

# move neighborhood column to the first column
york_fixed_columns = [york_onehot.columns[-1]] + list(york_onehot.columns[:-1])
york_onehot = york_onehot[york_fixed_columns]

york_onehot.head()

,Neighbourhood,Breakfast Spot,Bus Line,Check Cashing Service,Fast Food Restaurant,Field,Fried Chicken Joint,Grocery Store,Hockey Arena,Market,Park,Pizza Place,Sandwich Place,Trail,Women's Store
0,Caledonia-Fairbanks,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,Caledonia-Fairbanks,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,Caledonia-Fairbanks,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,Caledonia-Fairbanks,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,Caledonia-Fairbanks,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [130]:
york_onehot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 15 columns):
Neighbourhood            18 non-null object
Breakfast Spot           18 non-null uint8
Bus Line                 18 non-null uint8
Check Cashing Service    18 non-null uint8
Fast Food Restaurant     18 non-null uint8
Field                    18 non-null uint8
Fried Chicken Joint      18 non-null uint8
Grocery Store            18 non-null uint8
Hockey Arena             18 non-null uint8
Market                   18 non-null uint8
Park                     18 non-null uint8
Pizza Place              18 non-null uint8
Sandwich Place           18 non-null uint8
Trail                    18 non-null uint8
Women's Store            18 non-null uint8
dtypes: object(1), uint8(14)
memory usage: 476.0+ bytes


In [131]:
york_grouped = york_onehot.groupby('Neighbourhood').mean().reset_index()
york_grouped.head()

,Neighbourhood,Breakfast Spot,Bus Line,Check Cashing Service,Fast Food Restaurant,Field,Fried Chicken Joint,Grocery Store,Hockey Arena,Market,Park,Pizza Place,Sandwich Place,Trail,Women's Store
0,Caledonia-Fairbanks,0.00,0.00,0.0,0.2,0.000000,0.0,0.00,0.000000,0.2,0.4,0.00,0.0,0.000000,0.2
1,"Del Ray,Mount Dennis,Keelesdale,Silverthorn",0.00,0.00,0.2,0.4,0.000000,0.2,0.00,0.000000,0.0,0.0,0.00,0.2,0.000000,0.0
2,Humewood-Cedarvale,0.00,0.00,0.0,0.0,0.333333,0.0,0.00,0.333333,0.0,0.0,0.00,0.0,0.333333,0.0
3,"Runnymede,The Junction North",0.25,0.25,0.0,0.0,0.000000,0.0,0.25,0.000000,0.0,0.0,0.25,0.0,0.000000,0.0
4,Weston,0.00,0.00,0.0,0.0,0.000000,0.0,0.00,0.000000,0.0,1.0,0.00,0.0,0.000000,0.0


In [132]:
num_top_venues = 3

for hood in york_grouped['Neighbourhood']:
    print("----"+hood+"----")
    york_temp = york_grouped[york_grouped['Neighbourhood'] == hood].T.reset_index()
    york_temp.columns = ['venue','freq']
    york_temp = york_temp.iloc[1:]
    york_temp['freq'] = york_temp['freq'].astype(float)
    york_temp = york_temp.round({'freq': 2})
    print(york_temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Caledonia-Fairbanks----
                  venue  freq
0                  Park   0.4
1  Fast Food Restaurant   0.2
2                Market   0.2


----Del Ray,Mount Dennis,Keelesdale,Silverthorn----
                   venue  freq
0   Fast Food Restaurant   0.4
1  Check Cashing Service   0.2
2    Fried Chicken Joint   0.2


----Humewood-Cedarvale----
          venue  freq
0         Field  0.33
1  Hockey Arena  0.33
2         Trail  0.33


----Runnymede,The Junction North----
            venue  freq
0  Breakfast Spot  0.25
1        Bus Line  0.25
2   Grocery Store  0.25


----Weston----
            venue  freq
0            Park   1.0
1  Breakfast Spot   0.0
2        Bus Line   0.0




In [133]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [134]:
num_top_venues = 17

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
york_neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)

york_neighbourhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue


In [135]:
york_neighbourhoods_venues_sorted['Neighbourhood'] = york_grouped['Neighbourhood']

york_neighbourhoods_venues_sorted.head(2)

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue
0,Caledonia-Fairbanks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Del Ray,Mount Dennis,Keelesdale,Silverthorn",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [137]:
# set number of clusters
kclusters = 2

york_grouped_clustering = york_grouped.drop('Neighbourhood', 1)

# run k-means clustering
york_kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(york_grouped_clustering)

# check cluster labels generated for each row in the dataframe
york_kmeans.labels_[0:5] 

array([0, 1, 1, 1, 0], dtype=int32)

In [138]:
york_neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', york_kmeans.labels_)

york_merged = york_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
york_merged = york_merged.join(york_neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

york_merged


,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue
0,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M6M,York,"Del Ray,Mount Dennis,Keelesdale,Silverthorn",43.691116,-79.476013,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,M6C,York,Humewood-Cedarvale,43.693781,-79.428191,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,M6N,York,"Runnymede,The Junction North",43.673185,-79.487262,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,M9N,York,Weston,43.706876,-79.518188,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [139]:
# create map
york_map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(york_merged['Latitude'], york_merged['Longitude'], york_merged['Neighbourhood'], york_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(york_map_clusters)
       
york_map_clusters


In [140]:
york_map_clusters.save("york_map_clusters.html")


In [141]:
york_merged.loc[york_merged['Cluster Labels'] == 0, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue
0,York,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,York,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [142]:
#blue cluster
york_merged.loc[york_merged['Cluster Labels'] == 1, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue
1,York,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,York,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,York,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
